# Collecting data from ThermoML xml files


In [ ]:
import os
from ThermoML import ThermoMLBuilder
import polars as pl
import numpy as np
from importlib import reload
import glob
from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import CalcExactMolWt

thermoml_schema.py generator:


In [ ]:
# pyxbgen -u ThermoML/ThermoML.xsd -m ThermoML/thermoml_schema

In [ ]:
def RetrieveFiles():
    """
    Outputs a list of files path from ThermoML archive.

    Outputs
    ----------
    files: list(str)
        list of paths for each reference file
    """

    dir = os.getcwd()
    files = [os.path.join(dir, file) for file in glob.glob("ThermoML/*/*.xml")]
    return files

## Raw data


Collecting data from all files


In [ ]:
files = RetrieveFiles()
files[0]

In [ ]:
data = ThermoMLBuilder.build_dataset(files, "raw", executor="process", max_workers=12)